In [1]:
import pandas as pd
import numpy as np
import re
import string
#import spacy
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
df_news = pd.read_csv('stock_dataset/Combined_News_DJIA.csv')

In [3]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

In [4]:
def clean_text(text):
    text = str(text)#.lower()    # Making Text Lowercase
    text = re.sub('b\"|b\'|\\\\|\\\"', '', text) # remove the format that text starts with b" or b'
    text = re.sub('\[.*?\]', '', text) # remove text in [] brackets
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    # Clean contraction by using contraction mapping   
    for word in mapping.keys():
        if str(word) in text:
            text = text.replace(str(word), str(mapping[word]))
    #Remove Punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
 
    return text

def remove_space(text): 
    ## remove space that are too long
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = remove_space(text)
    return text

In [5]:
df_news_pro = df_news.copy()

for i in range(df_news_pro.shape[0]):
    for j in range(2, df_news_pro.shape[1]):
        df_news_pro.iloc[i,j] = text_preprocessing_pipeline(df_news.iloc[i,j])
df_news_pro

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity U N offici...,Russian tanks have entered South Ossetia whils...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,...,Georgia Invades South Ossetia if Russia gets i...,Al Qaeda Faces Islamist Backlash,Condoleezza Rice The US would not act to preve...,This is a busy day The European Union has appr...,Georgia will withdraw soldiers from Iraq to he...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a series ...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico s Kidnapping Surge
1,2008-08-11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S Ossetia blames...,...,Israel and the US behind the Georgian aggression,Do not believe TV neither Russian nor Georgian...,Riots are still going on in Montreal Canada be...,China to overtake US as largest manufacturer,War in South Ossetia,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over ...,Perhaps the question about the Georgia Russia ...,Russia is so much better at war,So this is what it is come to trading sex for ...
2,2008-08-12,0,Remember that adorable year old who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,Al Qa eda is losing support in Iraq because of...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO l...,Stratfor The Russo Georgian War and the Balanc...,I am Trying to Get a Sense of This Whole Georg...,...,U S troops still in Georgia did you know they ...,Why Russias response to Georgia was right,Gorbachev accuses U S of making a serious blun...,Russia Georgia and NATO Cold War Two,Remember that adorable year old who led your c...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS Asia Pacific Extinction by man not cl...
3,2008-08-13,0,U S refuses Israel weapons to attack Iran report,When the president ordered to attack Tskhinval...,Israel clears troops who killed Reuters cameraman,Britain s policy of being tough on drugs is po...,Body of year old found in trunk Latest ransom ...,China has moved million quake survivors into p...,Bush announces Operation Get All Up In Russia ...,Russian forces sink Georgian ships,...,Elephants extinct by,US humanitarian missions soon in Georgia if Ru...,Georgia s DDOS came from US sources,Russian convoy heads into Georgia violating truce,Israeli defence minister US against strike on ...,Gorbachev We Had No Choice,Witness Russian forces head towards Tbilisi in...,Quarter of Russians blame U S for conflict poll,Georgian president says US military will take ...,Nobel laureate Aleksander Solzhenitsyn accuses...
4,2008-08-14,1,All the experts admit that we should legalise ...,War in South Osetia pictures made by a Russian...,Swedish wrestler Ara Abrahamian throws away me...,Russia exaggerated the death toll in South Oss...,Missile That Killed Inside Pakistan May Have B...,Rushdie Condemns Random House s Refusal to Pub...,Poland and US agree to missle defense deal Int...,Will the Russians conquer Tblisi Bet on it no ...,...,Bank analyst forecast Georgian crisis days early,Georgia confict could s

In [14]:
# count words
word_count = []
for i in range(df_news_pro.shape[0]):
    for j in range(2, df_news_pro.shape[1]):
        word_count.append(len(str(df_news_pro.iloc[i,j]).split()))
        
print(min(word_count))
print(max(word_count))
print(np.mean(np.array(word_count)))

1
64
17.684725992961287


In [12]:
df_news_pro.iloc[0,2]

'Georgia downs two Russian warplanes as countries move to brink of war'

In [11]:
headlines = []
for row in range(df_news_pro.shape[0]):
    headlines.append(' '.join(str(x) for x in df_news_pro.iloc[row,2:]))

In [12]:
df_news_pro['combined_news'] = headlines

In [11]:
headlines[0]

'Georgia downs two Russian warplanes as countries move to brink of war BREAKING Musharraf to be impeached Russia Today Columns of troops roll into South Ossetia footage from fighting YouTube Russian tanks are moving towards the capital of South Ossetia which has reportedly been completely destroyed by Georgian artillery fire Afghan children raped with impunity U N official says this is sick a three year old was raped and they do nothing Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets Breaking Georgia invades South Ossetia Russia warned it would intervene on SO s side The enemy combatent trials are nothing but a sham Salim Haman has been sentenced to years but will be kept longer anyway just because they feel like it Georgian troops retreat from S Osettain capital presumably leaving several hundred people killed Did the U S Prep Georgia for War with Russia Rice Gives Green Light for Israel to Attack Iran Says U S has no veto over Israeli military ops

In [13]:
df_news_pro

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_news
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity U N offici...,Russian tanks have entered South Ossetia whils...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,...,Al Qaeda Faces Islamist Backlash,Condoleezza Rice The US would not act to preve...,This is a busy day The European Union has appr...,Georgia will withdraw soldiers from Iraq to he...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a series ...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico s Kidnapping Surge,Georgia downs two Russian warplanes as countri...
1,2008-08-11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S Ossetia blames...,...,Do not believe TV neither Russian nor Georgian...,Riots are still going on in Montreal Canada be...,China to overtake US as largest manufacturer,War in South Ossetia,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over ...,Perhaps the question about the Georgia Russia ...,Russia is so much better at war,So this is what it is come to trading sex for ...,Why wont America and Nato help us If they wont...
2,2008-08-12,0,Remember that adorable year old who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,Al Qa eda is losing support in Iraq because of...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO l...,Stratfor The Russo Georgian War and the Balanc...,I am Trying to Get a Sense of This Whole Georg...,...,Why Russias response to Georgia was right,Gorbachev accuses U S of making a serious blun...,Russia Georgia and NATO Cold War Two,Remember that adorable year old who led your c...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS Asia Pacific Extinction by man not cl...,Remember that adorable year old who sang at th...
3,2008-08-13,0,U S refuses Israel weapons to attack Iran report,When the president ordered to attack Tskhinval...,Israel clears troops who killed Reuters cameraman,Britain s policy of being tough on drugs is po...,Body of year old found in trunk Latest ransom ...,China has moved million quake survivors into p...,Bush announces Operation Get All Up In Russia ...,Russian forces sink Georgian ships,...,US humanitarian missions soon in Georgia if Ru...,Georgia s DDOS came from US sources,Russian convoy heads into Georgia violating truce,Israeli defence minister US against strike on ...,Gorbachev We Had No Choice,Witness Russian forces head towards Tbilisi in...,Quarter of Russians blame U S for conflict poll,Georgian president says US military will take ...,Nobel laureate Aleksander Solzhenitsyn accuses...,U S refuses Israel weapons to attack Iran repo...
4,2008-08-14,1,All the experts admit that we should legalise ...,War in South Osetia pictures made by a Russian...,Swedish wrestler Ara Abrahamian throws away me...,Russia exaggerated the death toll in South Oss...,Missile That Killed Inside Pakistan May Have B...,Rushdie Condemns Random House s Refusal to Pub...,Poland and US agree to missle defense deal Int...,Will the Russians conquer Tblisi Bet on it no ...,...,Georgia confict could set back Rus

In [78]:
sentences = df_news_pro.iloc[row_idx,2:-2].tolist()

# Cluster the sentences using k-means
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)

# Print the cluster labels for each sentence
for i, sentence in enumerate(sentences):
    print(f"Sentence '{sentence}' belongs to cluster {kmeans.labels_[i]}")

Sentence 'It s Official Ireland Makes Blasphemy Illegal Seriously Passed Wednesday legislation making blasphemy illegal with a Euro fine Police may also enter homes and confiscate blasphemous materials including books artwork cartoons of Mohammed whatever Book burnings next' belongs to cluster 3
Sentence 'Swine Flu only kills Fat people' belongs to cluster 4
Sentence 'Rude French are worst tourists' belongs to cluster 4
Sentence 'Murdoch yes he is the owner of Fox paid m to cover up illegal phone jacking' belongs to cluster 3
Sentence 'How the Irish Government manipulated the system to make sure the Blasphemy Bill would get through' belongs to cluster 3
Sentence 'Canada on the list of top carbon emitters refuses to change climate change plan despite fact that it falls short of the new G agreement Canada now takes US s old position among Bad Boy Nations' belongs to cluster 1
Sentence 'Right wing Jewish group trying to stop Israeli settlement building in the West Bank is equivalent to et

In [229]:
def check_important_news(df, K):
    """
    return: a list of number of the most important news (one number for each day)
    K: number of keywords used for filtering in a single day
    
    """
    length = []
    for i in range(df.shape[0]):
        vectorizer = TfidfVectorizer(stop_words ='english')
        # count当天的所有新闻 tfidf
        X = vectorizer.fit_transform(df.iloc[i, 2:27])
        keyword_sort_idx = np.argsort(X.toarray().sum(axis=0).tolist())
        #print("keyword",keyword_sort_idx)
        # 只关注前k个关键词
        keyword_sort = vectorizer.get_feature_names_out()[np.flip(keyword_sort_idx)[:K]]
        #print(keyword_sort)
        news_idx = []
        # 只保留含至少一个关键词的新闻
        for j in range(25):
            for keyword in keyword_sort:
                if keyword in df.iloc[i,2+j].lower():
                    news_idx.append(j)
                    break
        #print(len(news_idx))
        length.append(len(news_idx))
 
    return length #
        
            

In [231]:
# k = 15
# the first 15 keywords -> Related to at least 9 news items of the day
# min(check_important_news(df_news_pro2, 15))

9

In [233]:
# k = 15
# the first 15 keywords -> Related to an average of 17.5 news items of the day
# np.mean(check_important_news(df_news_pro2, 15))

17.499245852187027

In [234]:
def func2(df, K, n_clusters):
    """
    k: number of keywords used for filtering in a single day
    
    """
    for i in range(df.shape[0]):
        vectorizer = TfidfVectorizer(stop_words ='english')
        # count the tfidf of all news for the day
        X = vectorizer.fit_transform(df.iloc[i, 2:27])
        keyword_sort_idx = np.argsort(X.toarray().sum(axis=0).tolist())
        # only focus on the first k keywords
        keyword_sort = vectorizer.get_feature_names_out()[np.flip(keyword_sort_idx)[:K]]
        print(keyword_sort)
        news_idx = []
        # Keep only news containing at least one keyword
        keywords_num = {} # {j-th news : with how many keyword}
        for j in range(25):
            counter = 0
            for keyword in keyword_sort:
                if keyword in df.iloc[i,2+j].lower():
                    counter += 1
            keywords_num[j] = counter
            if counter > 0:
                news_idx.append(j)
        print(i, news_idx)
        # The day's top stories were clustered and de-duplicated, and only 3 were retained.
        kmeans = KMeans(n_clusters=n_clusters)
        kmeans.fit(X[news_idx,:])
        
        res = {}
        for j in range(len(news_idx)): 
            label = kmeans.labels_[j]
            for k in range(n_clusters): 
                if k==label:
                    # Clustering results keep the one with the most keywords for each clusters
                    if k not in res.keys():
                        res[k] = news_idx[j]
                    elif keywords_num[news_idx[j]] > keywords_num[res[k]]:
                        res[k] = news_idx[j]

        print(res)

        for k in res.keys():
            df.loc[i,k] = df.iloc[i,2+res[k]]
        
    print('Done!')
    return df

In [235]:
df_news_pro2 = df_news_pro.copy()

In [244]:
df_t = func2(df_news_pro2, 15, 3)

['georgia' 'south' 'ossetia' 'war' 'russia' 'russian' 'invades' 'breaking'
 'did' 'iran' 'countries' 'help' 'today' 'world' 'israeli']
0 [0, 1, 2, 3, 5, 6, 8, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 24]
{0: 0, 1: 6, 2: 10}
['russia' 'georgian' 'georgia' 'war' 'israeli' 'conflict' 'ossetia' 'help'
 'olympic' 'team' 'better' 'israel' 'aggression' 'south' 'new']
1 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 19, 20, 21, 22, 23]
{1: 15, 2: 2, 0: 19}
['georgia' 'war' 'russia' 'georgian' 'south' 'nato' 'ossetia' 'military'
 'fake' 'year' 'old' 'adorable' 'remember' 'did' 'israeli']
2 [0, 1, 4, 6, 7, 8, 9, 10, 11, 12, 15, 16, 18, 19, 20, 21, 22]
{1: 19, 2: 18, 0: 22}
['georgia' 'russia' 'iran' 'russian' 'israeli' 'strike' 'cnn' 'israel'
 'president' 'forces' 'georgian' 'extinct' 'elephants' 'choice'
 'gorbachev']
3 [0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
{2: 11, 0: 7, 1: 9}
['georgia' 'russia' 'south' 'war' 'georgian' 'russian' 'plans' 'b

{2: 23, 0: 19, 1: 10}
['world' 'blamed' 'pirates' 'afghanistan' 'says' 'green' 'canada'
 'children' 'real' 'atlas' 'dead' 'fucking' 'turnout' 'survey' 'global']
37 [0, 3, 4, 5, 8, 10, 11, 12, 14, 16, 18, 21, 22, 23, 24]
{2: 11, 1: 3, 0: 5}
['world' 'nuclear' 'russia' 'china' 'war' 'approves' 'deal' 'trade' 'new'
 'just' 'murder' 'real' 'atlas' 'capitals' 'international']
38 [1, 2, 5, 6, 9, 13, 15, 16, 17, 18, 19, 20, 21, 24]
{0: 6, 2: 2, 1: 15}
['afghanistan' 'dangerous' 'somalia' 'surge' 'settlers' 'juncture'
 'bailing' 'south' 'doesn' 'pakistan' 'monday' 'medina' 'kill' 'jewel'
 'strikes']
39 [0, 2, 4, 6, 8, 11, 12, 17, 19, 20, 21, 22, 23, 24]
{0: 0, 2: 6, 1: 8}
['military' 'says' 'government' 'europe' 'facing' 'germany' 'dark' 'dubai'
 'iceland' 'teacher' 'economic' 'taliban' 'pakistan' 'bankruptcy' 'iraq']
40 [0, 1, 2, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 17, 21, 22, 23]
{0: 1, 2: 4, 1: 17}
['says' 'russia' 'new' 'faces' 'war' 'taliban' 'iranian' 'terror' 'pepcon'
 'disaster' 'recen

{2: 5, 0: 2, 1: 8}
['pirate' 'ship' 'attack' 'sinking' 'russia' 'zimbabwe' 'acid'
 'schoolgirls' 'congo' 'shit' 'thai' 'sunken' 'owner' 'says' 'actually']
75 [1, 3, 5, 9, 11, 13, 14, 17, 18, 19, 21, 22, 24]
{0: 1, 2: 3, 1: 11}
['mumbai' 'thai' 'deadly' 'boat' 'indian' 'sank' 'pirate' 'navy' 'china'
 'attacked' 'terrorists' 'chinese' 'dead' 'people' 'shootings']
76 [2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 18, 21]
{2: 14, 0: 3, 1: 7}
['mumbai' 'attacks' 'hostages' 'israeli' 'airbus' 'crashes' 'hotel' 'taj'
 'finally' 'indian' 'test' 'regarding' 'google' 'map' 'deepening']
77 [0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 22, 23, 24]
{0: 7, 2: 17, 1: 24}
['mumbai' 'pirates' 'indian' 'police' 'sex' 'washington' 'blamed'
 'pakistan' 'declare' 'siege' 'giant' 'view' 'hate' 'mumbaikar' 'japanese']
78 [0, 3, 4, 5, 6, 7, 9, 12, 13, 16, 17, 18, 19, 20, 21, 23, 24]
{1: 9, 0: 5, 2: 13}
['mumbai' 'pakistan' 'china' 'years' 'free' 'terrorists' 'runs' 'works'
 'house' 'removed' 'says' 'fi

{1: 4, 0: 13, 2: 6}
['chinese' 'obama' 'guantanamo' 'shot' 'israeli' 'milk' 'death'
 'phosphorus' 'gazans' 'congress' 'israel' 'state' 'home' 'released'
 'teargas']
113 [0, 1, 2, 3, 4, 7, 8, 9, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24]
{1: 7, 0: 12, 2: 15}
['gaza' 'holocaust' 'palestinian' 'orders' 'pakistan' 'obama' 'appeal'
 'pm' 'says' 'attack' 'shocked' 'destruction' 'congo' 'gypsies' 'country']
114 [1, 2, 3, 4, 5, 8, 9, 12, 14, 15, 16, 17, 20, 21, 22, 23]
{2: 5, 1: 14, 0: 4}
['gaza' 'children' 'israel' 'israeli' 'pakistan' 'mugabe' 'zimbabwe'
 'danish' 'elections' 'angry' 'human' 'bank' 'mother' 'produce'
 'boycotting']
115 [0, 1, 2, 3, 4, 6, 7, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24]
{1: 1, 0: 4, 2: 6}
['gaza' 'israel' 'border' 'killed' 'using' 'army' 'leningrad' 'siege'
 'israeli' 'ban' 'torture' 'hate' 'strikes' 'clash' 'money']
116 [0, 1, 3, 4, 5, 7, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 23]
{2: 3, 0: 13, 1: 17}
['gaza' 'israeli' 'obama' 'crisis' 'israel' 'ki

{0: 1, 2: 2, 1: 23}
['gaza' 'israeli' 'war' 'soldiers' 'pope' 'israel' 'break' 'ranks'
 'troops' 'admit' 'china' 'france' 'recession' 'scenes' 'condoms']
152 [0, 1, 2, 5, 6, 8, 9, 11, 12, 13, 14, 16, 17, 19, 22]
{0: 19, 1: 1, 2: 2}
['eunuch' 'says' 'gaza' 'israel' 'obama' 'dead' 'people' 'perfect' 'heist'
 'rights' 'war' 'civilians' 'japanese' 'suicide' 'forest']
153 [0, 1, 3, 5, 6, 8, 11, 12, 15, 17, 18, 20]
{1: 1, 2: 12, 0: 8}
['gaza' 'israeli' 'says' 'afghanistan' 'north' 'korea' 'israel' 'war'
 'women' 'workers' 'religious' 'tigers' 'tamil' 'soldier' 'banks']
154 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 22, 23, 24]
{2: 3, 0: 13, 1: 2}
['china' 'new' 'global' 'violence' 'using' 'dollar' 'israel' 'court'
 'death' 'bans' 'youtube' 'israeli' 'hit' 'shovel' 'proceeded']
155 [1, 2, 4, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24]
{0: 15, 2: 9, 1: 11}
['youtube' 'president' 'china' 'israeli' 'year' 'death' 'bank' 'jewish'
 'israel' 'chinese' 'world' 'ar

...

{2: 3, 1: 11, 0: 23}
['just' 'new' 'says' 'china' 'pluto' 'attacks' 'government' 'party'
 'shenyang' 'pollution' 'officials' 'stop' 'facebook' 'toxic' 'health']
1827 [0, 2, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24]
{0: 0, 2: 4, 1: 20}
['germany' 'aid' 'military' 'years' 'british' 'billion' 'refugee'
 'protest' 'france' 'canada' 'entire' 'world' 'final' 'uk' 'australia']
1828 [0, 2, 3, 4, 5, 7, 9, 10, 11, 13, 14, 15, 17, 18, 20, 23, 24]
{1: 23, 2: 9, 0: 5}
['says' 'africa' 'new' 'isis' 'video' 'shows' 'north' 'country' 'oil'
 'russia' 'town' 'report' 'world' 'fuel' 'state']
1829 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 16, 18, 19, 20, 21, 22, 23, 24]
{0: 0, 1: 2, 2: 5}
['says' 'dead' 'homeopathy' 'confirm' 'jihadi' 'john' 'officials' 'island'
 'just' 'ukraine' 'brazil' 'prescribing' 'people' 'new' 'hong']
1830 [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 15, 19, 20, 21, 23, 24]
{1: 6, 0: 1, 2: 2}
['isis' 'paris' 'state' 'attacks' 'france' 'islamic' 'war' 'raqqa'
 'police' 'fr

{0: 15, 2: 7, 1: 8}
['saudi' 'isis' 'kuwait' 'latest' 'uk' 'executions' 'strong' 'malaysia'
 'doctors' 'new' 'jihadi' 'state' 'iran' 'mass' 'arabia']
1864 [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15, 18, 19, 20, 21, 23]
{0: 4, 1: 12, 2: 20}
['israel' 'china' 'saudi' 'paris' 'yemen' 'police' 'water' 'energy'
 'issue' 'sugar' 'cases' 'egypt' 'shuts' 'market' 'turkey']
1865 [0, 1, 4, 5, 6, 8, 9, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23]
{2: 0, 0: 1, 1: 12}
['state' 'iran' 'arabia' 'saudi' 'new' 'north' 'korea' 'says' 'bans' 'uk'
 'benefit' 'minister' 'china' 'years' 'control']
1866 [2, 3, 5, 6, 7, 9, 10, 11, 12, 14, 16, 18, 19, 21, 22, 23, 24]
{1: 9, 0: 21, 2: 12}
['el' 'chapo' 'amp' 'korea' 'says' 'french' 'caught' 'women' 'west'
 'children' 'debt' 'wants' 'tourists' 'north' 'aquarium']
1867 [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24]
{2: 8, 1: 9, 0: 6}
['says' 'sell' 'isis' 'rbs' 'putin' 'russia' 'istanbul' 'run' 'year' 'did'
 'bombs' 'time' 'pay' 'dani

{2: 5, 0: 3, 1: 4}
['says' 'syria' 'russia' 'claims' 'middle' 'people' 'winner' 'prize' 'day'
 'parliament' 'eu' 'pakistani' 'church' 'life' 'coffee']
1903 [1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 15, 16, 18, 19, 20, 21, 23]
{2: 1, 0: 12, 1: 3}
['military' 'year' 'spain' 'seizes' 'uniforms' 'isis' 'police' 'pay'
 'group' 'nuclear' 'use' 'russia' 'singapore' 'prices' 'food']
1904 [0, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 23, 24]
{0: 23, 1: 3, 2: 13}
['north' 'claim' 'korean' 'kill' 'pakistan' 'nuclear' 'report' 'gunmen'
 'ship' 'new' 'china' 'president' 'loonie' 'time' 'cents']
1905 [0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]
{0: 20, 1: 2, 2: 7}
['china' 'year' 'women' 'exports' 'government' 'migrants' 'obama' 'house'
 'meet' 'netanyahu' 'offer' 'white' 'bases' 'said' 'military']
1906 [0, 1, 2, 4, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 20, 24]
{1: 0, 0: 13, 2: 2}
['north' 'china' 'right' 'korea' 'nuclear' 'new' 'court' 'attacks' 'says'


{1: 2, 2: 11, 0: 5}
['isis' 'errors' 'state' 'million' 'russia' 'city' 'press' 'philippines'
 'sign' 'tests' 'mitsubishi' 'fuel' 'planet' 'greened' 'rise']
1941 [1, 2, 4, 5, 6, 7, 10, 12, 13, 16, 19, 20, 22, 23]
{2: 12, 0: 22, 1: 10}
['work' 'week' 'day' 'venezuela' 'says' 'rejects' 'orders' 'criticism'
 'police' 'government' 'national' 'south' 'notice' 'court' 'climate']
1942 [0, 2, 3, 5, 6, 7, 9, 10, 13, 14, 15, 16, 18, 20, 22, 23, 24]
{0: 20, 1: 2, 2: 6}
['red' 'weapons' 'korea' 'missile' 'destruction' 'nuclear' 'people'
 'global' 'space' 'oxygen' 'falling' 'oceans' 'rates' 'indian' 'satellite']
1943 [1, 3, 6, 7, 8, 9, 11, 12, 15, 16, 17, 18, 19, 23]
{2: 7, 1: 11, 0: 16}
['norway' 'people' 'says' 'germany' 'firm' 'chinese' 'entry' 'saudi'
 'prince' 'spacex' 'american' 'official' 'helicopter' 'carrying'
 'military']
1944 [1, 2, 3, 5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 19, 21, 24]
{0: 1, 2: 2, 1: 17}
['ttip' 'leaked' 'documents' 'puerto' 'eu' 'trade' 'deal' 'publishes'
 'says' 'china' '

{1: 10, 0: 7, 2: 11}
['year' 'country' 'environmental' 'canada' 'world' 'russian' 'says'
 'china' 'chinese' 'borders' 'robbed' 'rio' 'liesl' 'tesch' 'janeiro']
1980 [0, 1, 2, 3, 5, 6, 7, 8, 10, 12, 13, 14, 15, 17, 18, 20, 21, 23, 24]
{2: 0, 0: 3, 1: 14}
['police' 'company' 'french' 'new' 'government' 'korea' 'agency'
 'security' 'launches' 'indian' 'trump' 'tepco' 'fukushima' 'meltdown'
 'admits']
1981 [0, 2, 4, 6, 8, 9, 10, 11, 12, 14, 15, 17, 18, 19, 21, 22, 24]
{1: 9, 0: 24, 2: 10}
['india' 'china' 'report' 'europe' 'day' 'says' 'air' 'killed' 'tornado'
 'powerful' 'strengthened' 'merkel' 'nato' 'amid' 'uk']
1982 [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 15, 17, 18, 22, 23, 24]
{2: 22, 1: 10, 0: 24}
['brexit' 'eu' 'leave' 'referendum' 'votes' 'result' 'uk' 'calls' 'google'
 'britain' 'european' 'union' 'vote' 'nhs' 'gibraltar']
1983 [0, 1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23]
{0: 7, 2: 1, 1: 17}
['eu' 'brexit' 'leave' 'referendum' 'uk' 'second' 'following' 's

In [245]:
df_t

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top20,Top21,Top22,Top23,Top24,Top25,combined_news,1,0,2
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity U N offici...,Russian tanks have entered South Ossetia whils...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,...,Georgia will withdraw soldiers from Iraq to he...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a series ...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico s Kidnapping Surge,Georgia downs two Russian warplanes as countri...,Breaking Georgia invades South Ossetia Russia ...,Georgia downs two Russian warplanes as countri...,Rice Gives Green Light for Israel to Attack Ir...
1,2008-08-11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S Ossetia blames...,...,War in South Ossetia,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over ...,Perhaps the question about the Georgia Russia ...,Russia is so much better at war,So this is what it is come to trading sex for ...,Why wont America and Nato help us If they wont...,Israel and the US behind the Georgian aggression,War in South Ossetia,Jewish Georgian minister Thanks to Israeli tra...
2,2008-08-12,0,Remember that adorable year old who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,Al Qa eda is losing support in Iraq because of...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO l...,Stratfor The Russo Georgian War and the Balanc...,I am Trying to Get a Sense of This Whole Georg...,...,Remember that adorable year old who led your c...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS Asia Pacific Extinction by man not cl...,Remember that adorable year old who sang at th...,Remember that adorable year old who led your c...,Christopher King argues that the US and NATO a...,Russia Georgia and NATO Cold War Two
3,2008-08-13,0,U S refuses Israel weapons to attack Iran report,When the president ordered to attack Tskhinval...,Israel clears troops who killed Reuters cameraman,Britain s policy of being tough on drugs is po...,Body of year old found in trunk Latest ransom ...,China has moved million quake survivors into p...,Bush announces Operation Get All Up In Russia ...,Russian forces sink Georgian ships,...,Israeli defence minister US against strike on ...,Gorbachev We Had No Choice,Witness Russian forces head towards Tbilisi in...,Quarter of Russians blame U S for conflict poll,Georgian president says US military will take ...,Nobel laureate Aleksander Solzhenitsyn accuses...,U S refuses Israel weapons to attack Iran repo...,of CNN readers Russia s actions in Georgia jus...,Russian forces sink Georgian ships,US warns against Israeli plan to strike agains...
4,2008-08-14,1,All the experts admit that we should legalise ...,War in South Osetia pictures made by a Russian...,Swedish wrestler Ara Abrahamian throws away me...,Russia exaggerated the death toll in South Oss...,Missile That Killed Inside Pakistan May Have B...,Rushdie Condemns Random House s Refusal to Pub...,Poland and US agree to missle defense deal Int...,Will the Russians conquer Tblisi Bet on it no ...,...,Georgian TV reporter shot by Russian sniper du.

In [249]:
new_df = df_t[['Date', 'Label', 0, 1, 2]]
new_df

,Date,Label,0,1,2
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,Breaking Georgia invades South Ossetia Russia ...,Rice Gives Green Light for Israel to Attack Ir...
1,2008-08-11,1,War in South Ossetia,Israel and the US behind the Georgian aggression,Jewish Georgian minister Thanks to Israeli tra...
2,2008-08-12,0,Christopher King argues that the US and NATO a...,Remember that adorable year old who led your c...,Russia Georgia and NATO Cold War Two
3,2008-08-13,0,Russian forces sink Georgian ships,of CNN readers Russia s actions in Georgia jus...,US warns against Israeli plan to strike agains...
4,2008-08-14,1,War in South Osetia pictures made by a Russian...,Russia exaggerated the death toll in South Oss...,The US and Poland have agreed a preliminary de...
...,...,...,...,...,...
1984,2016-06-27,0,Boris Johnson says the UK will continue to int...,Richard Branson is calling on the UK governmen...,Scotland welcome to join EU Merkel ally says
1985,2016-06-28,1,CEO of the South African state broadcaster qui...,Ryanair will not deploy new aircraft on routes...,US Canada and Mexico pledge of power from clea...
1986,2016-06-29,1,Former Belgian Prime Minister ridicules Nigel ...,Explosion At Airport In Istanbul,BBC News Labour MPs pass Corbyn no confidence ...
1987,2016-06-30,1,Japan s top court has approved blanket surveil...,Groups urge U N to suspend Saudi Arabia from r...,UEFA says no minutes of silence for Istanbul v...


In [250]:
new_headlines = []
for row in range(new_df.shape[0]):
    new_headlines.append(' '.join(str(x) for x in new_df.iloc[row,2:]))

In [251]:
new_df['new_combined_news'] = new_headlines

/var/folders/1y/py9jvq8n72jg8pbt6zwtn9vr0000gn/T/ipykernel_75980/138874371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_combined_news'] = new_headlines


In [252]:
new_df

,Date,Label,0,1,2,new_combined_news
0,2008-08-08,0,Georgia downs two Russian warplanes as countri...,Breaking Georgia invades South Ossetia Russia ...,Rice Gives Green Light for Israel to Attack Ir...,Georgia downs two Russian warplanes as countri...
1,2008-08-11,1,War in South Ossetia,Israel and the US behind the Georgian aggression,Jewish Georgian minister Thanks to Israeli tra...,War in South Ossetia Israel and the US behind ...
2,2008-08-12,0,Christopher King argues that the US and NATO a...,Remember that adorable year old who led your c...,Russia Georgia and NATO Cold War Two,Christopher King argues that the US and NATO a...
3,2008-08-13,0,Russian forces sink Georgian ships,of CNN readers Russia s actions in Georgia jus...,US warns against Israeli plan to strike agains...,Russian forces sink Georgian ships of CNN read...
4,2008-08-14,1,War in South Osetia pictures made by a Russian...,Russia exaggerated the death toll in South Oss...,The US and Poland have agreed a preliminary de...,War in South Osetia pictures made by a Russian...
...,...,...,...,...,...,...
1984,2016-06-27,0,Boris Johnson says the UK will continue to int...,Richard Branson is calling on the UK governmen...,Scotland welcome to join EU Merkel ally says,Boris Johnson says the UK will continue to int...
1985,2016-06-28,1,CEO of the South African state broadcaster qui...,Ryanair will not deploy new aircraft on routes...,US Canada and Mexico pledge of power from clea...,CEO of the South African state broadcaster qui...
1986,2016-06-29,1,Former Belgian Prime Minister ridicules Nigel ...,Explosion At Airport In Istanbul,BBC News Labour MPs pass Corbyn no confidence ...,Former Belgian Prime Minister ridicules Nigel ...
1987,2016-06-30,1,Japan s top court has approved blanket surveil...,Groups urge U N to suspend Saudi Arabia from r...,UEFA says no minutes of silence for Istanbul v...,Japan s top court has approved blanket surveil...


In [253]:
def train_valid_split_data(df, train_ratio): # Date: 
    threshold = df.iloc[int(np.floor(df.shape[0] * train_ratio)), 0]
    df_train = df[df['Date'] < threshold]
    df_test = df[df['Date'] >= threshold]
    print(df_train.shape[0], df_test.shape[0])
    return df_train, df_test

In [254]:
df_train, df_valid_test = train_valid_split_data(new_df, 0.8)

1591 398


In [255]:
df_valid, df_test = train_valid_split_data(df_valid_test, 0.5)

199 199


In [256]:
print(df_train.shape)
print(df_train.iloc[0,0])
print(df_train.iloc[-1,0])

(1591, 6)
2008-08-08
2014-12-02


In [257]:
print(df_valid.shape)
print(df_valid.iloc[0,0])
print(df_valid.iloc[-1,0])

(199, 6)
2014-12-03
2015-09-17


In [258]:
print(df_test.shape)
print(df_test.iloc[0,0])
print(df_test.iloc[-1,0])

(199, 6)
2015-09-18
2016-07-01


In [260]:
df_train.to_csv('Combined_train_3news.csv', index=False)
df_valid.to_csv('Combined_valid_3news.csv', index=False)
df_test.to_csv('Combined_test_3news.csv', index=False)

In [ ]:
# df_train = pd.read_csv('../Combined_train.csv')
# df_valid = pd.read_csv('../Combined_valid.csv')
# df_test = pd.read_csv('../Combined_test.csv')